In [23]:
from sklearn.model_selection import StratifiedGroupKFold

In [18]:
from sklearn.model_selection import StratifiedKFold

In [36]:
sgkfold = StratifiedKFold(n_splits=4)

In [80]:
for trn_idx , val_idx in sgkfold.split(X=train_df['overview'].values,y=train_df['cat3'].values):
    break

c:\users\data\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


array([ 1512,  1552,  1705, ..., 16983, 16984, 16985])

In [53]:
splitter = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)

splits = []
for train_idx, test_idx in splitter.split(train_df['overview'], train_df['cat3']):
    splits.append((train_idx, test_idx))

c:\users\data\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


In [54]:
splits

[(array([    0,     1,     2, ..., 16982, 16984, 16985]),
  array([    4,     5,    12, ..., 16978, 16980, 16983])),
 (array([    0,     3,     4, ..., 16982, 16983, 16984]),
  array([    1,     2,     6, ..., 16963, 16972, 16985])),
 (array([    1,     2,     3, ..., 16980, 16983, 16985]),
  array([    0,    10,    11, ..., 16981, 16982, 16984])),
 (array([    0,     1,     2, ..., 16983, 16984, 16985]),
  array([    3,     7,     8, ..., 16973, 16975, 16979]))]

In [52]:
from torch.utils.data import Sampler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import BatchSampler

In [72]:
train_df

,id,img_path,overview,cat1,cat2,cat3,cat3_enc
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,120
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,8
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,118
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,118
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,118
...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,118
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,31
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,31
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",73


In [495]:
# sampler image id를 가지고 가져온다 dataloader 에 넣으면
class TextSampler(Sampler):
    def __init__(self,
                 # sample_idx,
                 data_source='./input/train.csv'):
        super().__init__(data_source)
        # self.sample_idx = sample_idx
        self.df = pd.read_csv(data_source)
        self.skf = StratifiedKFold(n_splits=4, shuffle=False)
    def __iter__(self):
         # overviews = self.df_images['id'].loc[self.sample_idx]
         for train_idx, test_idx in self.skf.split(self.df['overview'], self.df['cat3']):
            yield test_idx
        # return self.sample_idx
        # for idx in self.sample_idx:
            # yield idx
        # return iter(overviews)

    def __len__(self):
        return len(self.df)

In [ ]:
import torch
from sklearn.model_selection import StratifiedKFold

class StratifiedBatchSampler:
    """Stratified batch sampling
    Provides equal representation of target classes in each batch
    """
    def __init__(self, y, batch_size, shuffle=True):
        if torch.is_tensor(y):
            y = y.numpy()
        assert len(y.shape) == 1, 'label array must be 1D'
        n_batches = int(len(y) / batch_size)
        self.X = torch.randn(len(y),1).numpy()
        self.y = y
        self.shuffle = shuffle

    def __iter__(self):
        if self.shuffle:
            self.skf.random_state = torch.randint(0,int(1e8),size=()).item()
        for train_idx, test_idx in self.skf.split(self.X, self.y):
            yield test_idx

    def __len__(self):
        return len(self.y)

In [197]:
class CateDataset(Dataset):
    """
    데이터셋에서 학습에 필요한 형태로 변환된 샘플 하나를 반환
    """
    def __init__(self, df_data,   tokens_max_len=64, type_vocab_size=30):
        """
        :param df_data: 상품 타이틀 , 카테고리 등의 정보를 가지는 데이터프레임
        :param img_h5_path:  img_feat가 저장돼 있는 h5파일의 경로
        :param tokens_max_len: tokens 의 최대 길이 , 상품명의 tokens 가 이 이상이면 잘라서 버림
        :param type_vocab_size: 타입 사전의 크기
        """
        # 작문 평가방법처럼 해보기 , 카카오 개선사항과 작문평가 kaggle 참조
        self.tokens = df_data['overview'].values # 전처리된 상품명
        self.tokens_max_len = tokens_max_len
        self.df_data = df_data
        self.labels = df_data[['cat3']].values
        self.p = re.compile('▁[^▁]+') #  기호를 기준으로 나누기 위한 컴파일 정규식 _ 기호와 다름 ▁ 이것으로 사용
        self.type_vocab_size = type_vocab_size

    def __getitem__(self, idx):
        """
        데이터셋에서 idx에 대응되는 샘플을 변환하여 반환

        :param item:
        :return:
        """
        # if idx >= len(self.tokens):
        #     raise StopIteration

        # idx 에 해당하는 상품명 가져오기 , 상품명은 문자열로 저장돼 있음


        tokens = self.tokens[idx]
        return tokens
        # if not isinstance(tokens, str):
        #     tokens = ''
        # # 상품명을 _기호를 기준으로 분리하여 파이썬 리스트로 저장
        # # "_직소퍼즐 _1000 조각 _바다 거북 의 _여행 _pl 12 75" =>
        # # ["_직소퍼즐" , "_1000조각", "_바다 거북 의", "_여행","_pl 12 75"]
        # tokens = self.p.findall(tokens)
        #
        # token_types = [type_id for type_id , word in enumerate(tokens) for _ in word.split()]
        # tokens=  " ".join(tokens) # _기호로 분리되기 전의 원래의 tokens로 되돌림
        #
        # token_ids = [self.token2id[tok] if tok in self.token2id else 0 for tok in tokens.split()]
        #
        # # token_ids 의 길이가 max_len보다 길다면 잘라서 버린다
        # if len(token_ids) > self.tokens_max_len:
        #     token_ids = token_ids[:self.tokens_max_len]
        #     token_types = token_types[:self.tokens_max_len]
        #
        # # token_ids의 길이가 max_len보다 짧으면 짧은 만큼 PAD 값을 0으로 채워 넣음
        # # token_ids 중 값이 있는 곳은 1, 그 외는 0으로 채운 token_mask 생성
        # token_mask = [1] * len(token_ids)
        # token_pad = [0] * (self.tokens_max_len - len(token_ids))
        # token_ids += token_pad
        # token_mask += token_pad
        # token_types += token_pad # max_len 보다 짧은 만큼 PAD 추가
        #
        #
        # token_ids = torch.LongTensor(token_ids)
        # token_mask = torch.LongTensor(token_mask)
        # token_types = torch.LongTensor(token_types)
        #
        # # 카카오에서 진행한 개선 과정도 포함시키기
        # # token_type의 타입 인덱스의 숫자 크기가 type_vocab_size보다 작도록 바꿈
        # token_types[token_types >= self.type_vocab_size] = self.type_vocab_size-1
        #
        # # 안정적으로 img_feat를 가져오려면 아래처럼 매번 h5py.File 호출필요
        # with h5py.File(self.img_h5_path, 'r') as img_feats:
        #     img_feat = img_feats['img_feat'][self.img_indices[idx]]
        # #print(self.df_data.iloc[idx])
        # #print(self.df_data.iloc[0])
        # #print(self.img_indices[0])
        #
        # #img_feat = img_feat.view(1,-1)
        # # 대/중/소/세 레이블 준비
        # label = self.labels[idx]
        # label = torch.LongTensor(label )
        # #
        #
        # # 갑자기 batched index가 들어오는 경우가 있다 그건 문제안됨
        # # 크게 3가지 텍스트 입력, 이미지 입력 , 레이블을 반환
        # return token_ids , token_mask , token_types, img_feat, label

    def __len__(self):
        """
          tokens의 개수를 반환한다. 즉, 상품명 문장의 개수를 반환한다.
        """
        return len(self.tokens)

In [ ]:
#bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer,vocab,lower=False)

In [223]:
from transformers import AutoTokenizer

In [224]:
tok = AutoTokenizer.from_pretrained('klue/roberta-large')

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [482]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx , label_idx, bert_tokenizer, max_len):
    # transform = nlp.data.BERTSentenceTransform(
    #     bert_tokenizer, max_seq_length=max_len, pad=pad,pair=pair
  # bert_tokenizer 로 transform
    self.sentences = [bert_tokenizer.encode_plus(i[sent_idx],max_length=max_len,padding='max_length',truncation=True) for i in dataset.values]
    self.labels = [np.int32(i[label_idx]) for i in dataset.values]
  def __getitem__(self, i):
    # output,output2,output3 = self.sentences[0]['input_ids'],self.sentences[0]['token_type_ids'],self.sentences[0]['attention_mask']

    return self.sentences[i]['input_ids']
    # print(self.sentences[i] + (self.labels[i]))
    # return (self.sentences[i] + (self.labels[i],))
  def __len__(self):
    return (len(self.labels))

In [413]:
def create_split_loaders(dataset, split,   batch_size):
    train_folds_idx = split[0]
    valid_folds_idx = split[1]
    train_sampler = ImageSampler(train_folds_idx)
    valid_sampler = ImageSampler(valid_folds_idx)
    train_loader = DataLoader(dataset, batch_sampler=train_sampler)
    valid_loader = DataLoader(dataset, batch_sampler=valid_sampler)
    return (train_loader, valid_loader)

In [414]:
train_folds_idx = trn_idx
valid_folds_idx = val_idx

In [496]:
train_folds_idx

array([ 1512,  1552,  1705, ..., 16983, 16984, 16985])

In [497]:
train_sampler = TextSampler()

In [481]:
for tr_idx ,va_idx in train_sampler.skf.split(train_df['overview'], train_df['cat3']):
    train_df.iloc[tr_idx]
    break

c:\users\data\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


In [483]:
bert_df = BERTDataset(input_df, 0,1 , tok ,64)

In [485]:
train_loader = DataLoader(bert_df, batch_sampler=train_sampler,batch_size=1,shuffle=False)

In [501]:
for data in train_loader:
    break

c:\users\data\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


In [502]:
data

[tensor([0, 0, 0,  ..., 0, 0, 0]),
 tensor([ 1282,  4619, 23236,  ...,  4041, 14900, 10427]),
 tensor([ 2283, 16781, 14908,  ..., 14223,   879, 29047]),
 tensor([2421, 2067, 3271,  ..., 2079, 2199, 2053]),
 tensor([2073, 1078, 2588,  ..., 3824, 2154, 2376]),
 tensor([ 5574,  2116, 12799,  ...,  3877,  2073,  2021]),
 tensor([ 2470,  2460,  2259,  ..., 10697,  6273,  2259]),
 tensor([1264, 2170, 8680,  ..., 3751, 2470, 4407]),
 tensor([ 6233,  1513,  5129,  ...,  6233, 16842,  7271]),
 tensor([ 4973,  2259,  2154,  ..., 12325,  2145, 14659]),
 tensor([ 2097,  7802,  2069,  ..., 11830,  6701,    11]),
 tensor([ 2283,  6233,  3752,  ...,  7138,  2470, 21512]),
 tensor([ 2052,  4672, 11187,  ...,  2069,  5008,  1381]),
 tensor([8808, 2021, 7045,  ..., 3691, 8185, 2318]),
 tensor([ 2097,  3801, 19521,  ..., 31302,  2069,  2057]),
 tensor([2331, 2827, 3704,  ..., 5418, 7492, 2052]),
 tensor([ 6233, 28674,  2205,  ...,  2173,  5008,  2203]),
 tensor([12168,    18,  2259,  ...,  4584,  2189,  

In [367]:
# sentences = [tok.encode_plus(i[0],max_length=64) for i in input_df.values]

In [421]:
# len(train_loader.dataset.sentences[4]['input_ids'])

64

In [ ]:
# [tok.encode_plus(i[0],max_length=64,padding='max_length',truncation=True) for i in input_df.values]

In [434]:
# [tok.encode_plus(i[0],max_length=64,padding='max_length',truncation=True) for i in input_df.values]

In [432]:
# # for data in train_loader:
#     print(data)
#     break

[tensor([0, 0, 0,  ..., 0, 0, 0]), tensor([ 1282,  4619, 23236,  ...,  4041, 14900, 10427]), tensor([ 2283, 16781, 14908,  ..., 14223,   879, 29047]), tensor([2421, 2067, 3271,  ..., 2079, 2199, 2053]), tensor([2073, 1078, 2588,  ..., 3824, 2154, 2376]), tensor([ 5574,  2116, 12799,  ...,  3877,  2073,  2021]), tensor([ 2470,  2460,  2259,  ..., 10697,  6273,  2259]), tensor([1264, 2170, 8680,  ..., 3751, 2470, 4407]), tensor([ 6233,  1513,  5129,  ...,  6233, 16842,  7271]), tensor([ 4973,  2259,  2154,  ..., 12325,  2145, 14659]), tensor([ 2097,  7802,  2069,  ..., 11830,  6701,    11]), tensor([ 2283,  6233,  3752,  ...,  7138,  2470, 21512]), tensor([ 2052,  4672, 11187,  ...,  2069,  5008,  1381]), tensor([8808, 2021, 7045,  ..., 3691, 8185, 2318]), tensor([ 2097,  3801, 19521,  ..., 31302,  2069,  2057]), tensor([2331, 2827, 3704,  ..., 5418, 7492, 2052]), tensor([ 6233, 28674,  2205,  ...,  2173,  5008,  2203]), tensor([12168,    18,  2259,  ...,  4584,  2189,  4193]), tensor([ 

c:\users\data\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


In [165]:
train_sampler = TextSampler(trn_idx)
valid_sampler = TextSampler(val_idx)

In [198]:
# import re
# train_data = CateDataset(train_df)

In [199]:
# train_loader = DataLoader(train_data, batch_sampler=train_sampler)


In [201]:
# for data in train_loader:
#     print(data)
#     break

T


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [169]:
for data in train_loader:
    break

KeyError: 'T'

In [ ]:
valid_loader = DataLoader(dataset, batch_sampler=valid_sampler)
return (train_loader, valid_loader)

In [94]:
for d in iter(train_df['id'].loc[trn_idx]):
    print(d)
    break

TRAIN_01512


In [98]:
for data in train_sampler:
    break

In [99]:
data

'T'

In [86]:
for data in train_loader:
    break

KeyError: 'T'

In [ ]:
data

In [ ]:
valid_loader = DataLoader(train, batch_sampler=valid_sampler)

In [57]:
train_sampler = ImageSampler(trn_idx)

In [70]:

train_loader = DataLoader(train_df, batch_sampler=train_sampler)

In [71]:
for data in train_loader:
    print(train_loader)

KeyError: '경'

In [69]:
train_df

,id,img_path,overview,cat1,cat2,cat3,cat3_enc
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,120
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,8
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,118
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,118
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,118
...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,118
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,31
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,31
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",73


In [66]:
data

'경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 2016년 대중제로 전환하여 재개장했다. 총 부지 약 980,,000㎡에 전장 6,607m에 18홀 파 72이다. Lake 코스와 Mountain 코스가 있다. 미국 100대 골프 코스 설계자인 짐 파지오가 아마추어에게는 쉽고 프로골퍼에게는 어렵게 설계했다고 한다. 가까이에 뉴스프링빌CC, 써닝포인트CC, 비에이비스타CC, 덕평CC 등의 골프장이 있다.'

In [76]:
def get_all_split_loaders(dataset, cv_splits, aug_count=5, batch_size=30):
    """Create DataLoaders for each split.

    Keyword arguments:
    dataset -- Dataset to sample from.
    cv_splits -- Array containing indices of samples to
                 be used in each fold for each split.
    aug_count -- Number of variations for each sample in dataset.
    batch_size -- batch size.

    """
    split_samplers = []

    for i in range(len(cv_splits)):
        split_samplers.append(
            create_split_loaders(dataset,
                                 cv_splits[i],
                                 aug_count,
                                 batch_size)
        )
    return split_samplers

In [ ]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset = Dataset(transform=transform, device=device)

In [ ]:
dataloaders = get_all_split_loaders(dataset, splits, aug_count=5, batch_size=10)

In [50]:
splits

[(array([    0,     1,     2, ..., 16982, 16984, 16985]),
  array([    4,     5,    12, ..., 16978, 16980, 16983])),
 (array([    0,     3,     4, ..., 16982, 16983, 16984]),
  array([    1,     2,     6, ..., 16963, 16972, 16985])),
 (array([    1,     2,     3, ..., 16980, 16983, 16985]),
  array([    0,    10,    11, ..., 16981, 16982, 16984])),
 (array([    0,     1,     2, ..., 16983, 16984, 16985]),
  array([    3,     7,     8, ..., 16973, 16975, 16979]))]

In [34]:
val_idx

array([0])

In [22]:
trn_idx

array([ 1512,  1552,  1705, ..., 16983, 16984, 16985])